<a href="https://colab.research.google.com/github/mostafa-ja/Data-Privacy/blob/main/Preserving_Data_Privacy_in_Deep_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
import random
from tqdm import tqdm
import numpy as np
import torch, torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data.dataset import Dataset 
torch.backends.cudnn.benchmark=True

In [3]:
##### Hyperparameters for federated learning #########
num_clients = 20
num_selected = 6
num_rounds = 150
epochs = 5
batch_size = 32



```
 generator = torch.Generator().manual_seed(42)
 random_split(range(10), [3, 7], generator=generator)

 train_data.data.shape[0]
```



In [12]:

#############################################################
##### Creating desired data distribution among clients  #####
#############################################################

# Image augmentation 
transform_train = transforms.Compose([
    transforms.RandomCrop(32,padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

# Loading CIFAR10 using torchvision.datasets
train_data = datasets.CIFAR10('./train_data',train=True,download=True,
                              transform=transform_train)

# Dividing the training data into num_clients, with each client having equal number of images
train_data_split = torch.utils.data.random_split(train_data,[int(train_data.data.shape[0]/num_clients) for _ in range(num_clients) ])

# Creating a pytorch loader for a Deep Learning model
train_loader = [torch.utils.data.DataLoader(x,batch_size=batch_size,shuffle=True) for x in train_data_split ]

# Normalizing the test images
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

# Loading the test iamges and thus converting them into a test_loader
test_data = datasets.CIFAR10('./test_data',train=False,download=True,transform=transform_test)
test_loader = torch.utils.data.DataLoader(test_data,batch_size=batch_size,shuffle=True)

Files already downloaded and verified


  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./test_data/cifar-10-python.tar.gz to ./test_data
